$$
\begin{align}
\frac{\partial\mathbf{u}}{\partial t}+\left(\mathbf{u}\cdot\nabla\right)\mathbf{u}&=-\frac{1}{\rho}\nabla p+\nu \nabla^2 \mathbf{u} + \frac{1}{\rho}\mathbf{f} ,\tag{1}\\
\nabla\cdot\mathbf{u}&=0.\tag{2}
\end{align}
$$

https://ocw.mit.edu/courses/2-20-marine-hydrodynamics-13-021-spring-2005/dc382bb85a7a4f14f71edc7f95393365_lecture9.pdf

In [2]:
from torchfsm.operator import Laplacian,Operator,VorticityConvection
from typing import Optional

def NavierStokesVorticity(force:Optional[Operator]=None,Re=100)->Operator:
    ns_vorticity=-VorticityConvection() + 1/Re*Laplacian()+force
    if force is not None:
        ns_vorticity+=force
    return ns_vorticity

In [3]:
import torch
from torchfsm.mesh import MeshGrid
from torchfsm.plot import plot_traj
from torchfsm.field import kolm_force,diffused_noise
from torchfsm.traj_recorder import AutoRecorder,IntervalController
device='cuda' if torch.cuda.is_available() else 'cpu'
L=torch.pi*2; N=128; 

In [4]:
mesh=MeshGrid([(0,L,N),(0,L,N)],device=device)
x,y=mesh.bc_mesh_grid()
kolm=NavierStokesVorticity(kolm_force(y),Re=100)
u0=diffused_noise(mesh,device=device,n_batch=2)
traj=kolm.integrate(
        u_0=u0,
        mesh=mesh,
        dt=0.5/50,
        step=100*50,
        trajectory_recorder=AutoRecorder(IntervalController(interval=50))
    )

In [5]:
plot_traj(traj)

In [6]:
from torchfsm.operator.dedicated._vorticity_convection import Vorticity2Velocity

v2v=Vorticity2Velocity()
ori_shape=traj.shape
traj=traj.view(-1,*ori_shape[2:])
velocity=v2v(traj,mesh=mesh).view(*ori_shape[0:2],2,*ori_shape[3:])

In [7]:
plot_traj(velocity,channel_names=['$u_x$','$u_y$'])

In [8]:
from torchfsm.operator import Div

div=Div()
div(velocity[1],mesh=mesh).pow(2).mean()

tensor(6.6694e-11, device='cuda:0')